<a href="https://colab.research.google.com/github/IlyaZelinskiy/Wav2vec_CFT_SHIFT_LAB/blob/main/eng_to_ipa_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Функция для того, чтобы узнать что за видокарта у нас подключена

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 28 14:45:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install huggingface-hub==0.11.1
!pip install eng-to-ipa 
from huggingface_hub import notebook_login

notebook_login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
repo_name = "wav2vec2-base-timit-demo-google-colab"

In [3]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0

In [4]:
%%capture
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

In [5]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [19]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [22]:
timit = timit.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [23]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [25]:
vocabs = timit.map(extract_all_phonems, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
!pip install eng-to-ipa 
import eng_to_ipa as p

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
def extract_all_phonems(batch):
    all_text = p.convert(batch['text'])
    all_text = " ".join(all_text)
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [26]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [30]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

In [28]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

46

In [31]:
vocab_dict = {
  "'": 29,
  "[PAD]": 43,
  "[UNK]": 42,
  "a": 32,
  "b": 15,
  "c": 9,
  "d": 12,
  "e": 25,
  "f": 24,
  "g": 16,
  "h": 34,
  "i": 13,
  "j": 30,
  "k": 22,
  "l": 20,
  "m": 23,
  "n": 11,
  "o": 6,
  "p": 38,
  "q": 39,
  "r": 28,
  "s": 3,
  "t": 35,
  "u": 4,
  "v": 7,
  "w": 41,
  "x": 19,
  "y": 1,
  "z": 17,
  "|": 27,
  "æ": 2,
  "ð": 36,
  "ŋ": 10,
  "ɑ": 21,
  "ɔ": 37,
  "ə": 5,
  "ɛ": 18,
  "ɪ": 40,
  "ʃ": 31,
  "ʊ": 33,
  "ʒ": 14,
  "ʤ": 26,
  "ʧ": 0,
  "θ": 8
}

In [32]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [33]:
from transformers import Wav2Vec2CTCTokenizer
!pip install jiwer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
tokenizer.push_to_hub(repo)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:42: FutureWarning: Deprecated positional argument(s) used in 'create_repo': pass token='wav2vec2-base-timit-demo-google-colab' as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  FutureWarning,


TypeError: ignored

In [34]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [35]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Функция для прослушивания аудио

In [36]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=False, rate=16000)

a voice spoke nearathand   


In [37]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 1680
    })
})

Наконец, мы можем обработать набор данных в формате, ожидаемом моделью для обучения. Мы будем использовать функцию map(...).

Во-первых, мы загружаем и передискретизируем аудиоданные, просто вызывая команду batch["audio"]. Во-вторых, мы извлекаем input_values ​​из загруженного аудиофайла. В нашем случае Wav2Vec2Processor только нормализует данные. Однако для других моделей речи этот шаг может включать более сложное извлечение признаков, например извлечение признаков Log-Mel. В-третьих, мы кодируем транскрипции для обозначения идентификаторов.

Примечание. Эта функция сопоставления является хорошим примером того, как следует использовать класс Wav2Vec2Processor. В «нормальном» контексте вызов process(...) перенаправляется на метод вызова Wav2Vec2FeatureExtractor. Однако при заключении процессора в контекст as_target_processor тот же метод перенаправляется на метод вызова Wav2Vec2CCTTokenizer.

In [38]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(p.convert(batch["text"])).input_ids
    return batch

In [19]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [39]:
dataset = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

In [42]:
timit['train']['text'][0]

'would such an act of refusal be useful   '

In [32]:
dataset['train']['labels'][0]

[12,
 22,
 11,
 10,
 17,
 4,
 24,
 11,
 15,
 26,
 4,
 14,
 3,
 4,
 14,
 15,
 9,
 4,
 22,
 16,
 4,
 1,
 5,
 16,
 11,
 24,
 14,
 10,
 4,
 6,
 5,
 4,
 11,
 24,
 5,
 16,
 11,
 10,
 4,
 4]

In [ ]:
p.convert(timit['train']['text'][0])

'wʊd səʧ ən ækt əv rɪfˈjuzəl bi ˈjusfəl'

In [33]:
label_features = [{"input_ids": dataset['train']['labels']} for feature in dataset]

In [ ]:
label_features

Отфильтруем записи более 4 сек

Длинные входные последовательности требуют много памяти. Поскольку Wav2Vec2 основан на внутреннем внимании, требования к памяти масштабируются квадратично с длиной входных данных для длинных входных последовательностей

In [43]:
max_input_length_in_sec = 4.0
dataset["train"] = dataset["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/5 [00:00<?, ?ba/s]

Данные обработаны, так что мы готовы приступить к настройке обучения. Мы будем использовать Trainer 🤗, для которого нам по существу нужно сделать следующее:

Определите сборщик данных. В отличие от большинства моделей NLP, Wav2Vec2 имеет намного большую входную длину, чем выходную. Например, выборка входной длины 50000 имеет выходную длину не более 100. Учитывая большие размеры входных данных, гораздо эффективнее динамически дополнять обучающие пакеты, что означает, что все обучающие выборки должны быть дополнены только самой длинной выборкой в ​​их партия и не общая самая длинная выборка. Поэтому для тонкой настройки Wav2Vec2 требуется специальный сортировщик данных заполнения, который мы определим ниже.

Метрика оценки. Во время обучения модель должна оцениваться по частоте ошибок в словах. Мы должны определить функцию calculate_metrics соответственно.

Загрузите предварительно обученную контрольную точку. Нам нужно загрузить предварительно обученную контрольную точку и правильно настроить ее для обучения.

Определите конфигурацию обучения.

После тонкой настройки модели мы правильно оценим ее на тестовых данных и убедимся, что она действительно научилась правильно транскрибировать речь.

In [44]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    -Сопоставитель данных, который будет динамически дополнять полученные входные данные.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
            -Процессор, используемый для обработки данных.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            -
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided). - Дополнение к самой длинной последовательности в пакете (или без заполнения, если только одна
              последовательность, если она указана).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided. - Дополнить до максимальной длины, указанной аргументом :obj:`max_length`, или до
              максимальная приемлемая длина входных данных для модели, если этот аргумент не указан.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [45]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [46]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [48]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [49]:
from transformers import AutoProcessor, AutoModelForCTC

#processor = AutoProcessor.from_pretrained("speech31/wav2vec2-large-english-TIMIT-phoneme_v3")

model = AutoModelForCTC.from_pretrained("speech31/wav2vec2-large-english-TIMIT-phoneme_v3")

Downloading:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [ ]:
from transformers import AutoProcessor, AutoModelForCTC

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-xls-r-phoneme-300m-tr", ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id)

In [27]:

from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_hid.weight', 'project_q.weight', 'quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
model

In [29]:
model.freeze_feature_encoder()

In [50]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [51]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [ ]:
trainer.train().cuda()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3978
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14940


Step,Training Loss,Validation Loss,Wer
500,0.493500,0.383595,0.357246
1000,0.397800,0.428570,0.314865
1500,0.441800,0.506305,0.340569
2000,0.380100,0.429337,0.329888


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8
Saving model checkpoint to wav2vec2-base-timit-demo-google-colab/checkpoint-500
Configuration saved in wav2vec2-base-timit-demo-google-colab/checkpoint-500/config.json
Model weights saved in wav2vec2-base-timit-demo-google-colab/checkpoint-500/pytorch_model.bin
Feature extractor saved in wav2vec2-base-timit-demo-google-colab/checkpoint-500/preprocessor_config.json
Deleting older checkpoint [wav2vec2-base-timit-demo-google-colab/checkpoint-5000] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,

In [ ]:
trainer

AttributeError: ignored

In [33]:
trainer.push_to_hub('wav2vec2-base-timit-demo-google-colab')

HTTPError: ignored

In [42]:
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")

https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/preprocessor_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpu5jyj32r


Downloading:   0%|          | 0.00/215 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/preprocessor_config.json in cache at /root/.cache/huggingface/transformers/17fa07f6e2f88edf24c71037bed1b7cf0cf9acfef7d3353afb589689627542c1.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
creating metadata file for /root/.cache/huggingface/transformers/17fa07f6e2f88edf24c71037bed1b7cf0cf9acfef7d3353afb589689627542c1.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
loading feature extractor configuration file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/17fa07f6e2f88edf24c71037bed1b7cf0cf9acfef7d3353afb589689627542c1.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_s

Downloading:   0%|          | 0.00/217 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/7bf2e07c1c7ed7e3d055d87aa6bd33c75e6883d98029ae3744fed6bea9e35095.59710b1a6a5501d31e746b6e464f5c44de3e55a58f80634196025936683a68a9
creating metadata file for /root/.cache/huggingface/transformers/7bf2e07c1c7ed7e3d055d87aa6bd33c75e6883d98029ae3744fed6bea9e35095.59710b1a6a5501d31e746b6e464f5c44de3e55a58f80634196025936683a68a9
https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/vocab.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp5t08x20v


Downloading:   0%|          | 0.00/268 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/f739813380b0880ba302557f0f990f0f64e8efe81fcc91a4fce08206a91ee215.c44a036121683e19f1a033a1765d4344364cb4633a6cbf5cf51b153f2388abb1
creating metadata file for /root/.cache/huggingface/transformers/f739813380b0880ba302557f0f990f0f64e8efe81fcc91a4fce08206a91ee215.c44a036121683e19f1a033a1765d4344364cb4633a6cbf5cf51b153f2388abb1
https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp6pxgaat9


Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/c239f3738252f8d8e83d9813ea82866aeec476c467a008db86229f4edc6490b0.a21d51735cf8667bcd610f057e88548d5d6a381401f6b4501a8bc6c1a9dc8498
creating metadata file for /root/.cache/huggingface/transformers/c239f3738252f8d8e83d9813ea82866aeec476c467a008db86229f4edc6490b0.a21d51735cf8667bcd610f057e88548d5d6a381401f6b4501a8bc6c1a9dc8498
loading file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/f739813380b0880ba302557f0f990f0f64e8efe81fcc91a4fce08206a91ee215.c44a036121683e19f1a033a1765d4344364cb4633a6cbf5cf51b153f2388abb1
loading file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/7bf2e07c1c7ed7e3d055d87aa6b

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab").cuda()

In [51]:
trainer.save_model("demo")

Saving model checkpoint to demo
Configuration saved in demo/config.json
Model weights saved in demo/pytorch_model.bin
Feature extractor saved in demo/preprocessor_config.json


In [34]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [37]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [44]:
results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

0ex [00:00, ?ex/s]

In [45]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 2.980


In [46]:
show_random_elements(results)

,pred_str,text
0,what shall these effects be,n tmiy tllim uyuiuhhufmyicui
1,his sholder felt as if it were broken,kyiy jglouxihulmityikhikminuxuicxjsupi
2,the morning juw on the spihter weab glistened in the sun,m uiwjxpkpqiounijpim uiy'kouxinuciqlkymupuoikpim uiygpi
3,that noys problem groes more innoring each day,m tmipjkyui'xjcluwiqxjnyiwjxuitppjrkpqiutf iotri
4,granmother outgrouwh her upbringing in petycoats,qxtpowjm uxijgmqxuni uxig'cxkpqkpqikpi'ummkfjtmyi
5,he admitted his behavior and defended it,uitowkmmuoi kyicu tdkjxitpoiouhupouoikmi
6,the fog preovented them from ariving on time,m uihjqi'xudupmuoim uwihxjwitxxkdkpqijpimkwui
7,rob sat by the pond and sketch the stray gooase,xjciytmicrim ui'jpoitpoiysumf uoim uiymxtriquuyui
8,jonclean shellfish for a living,bj piflutpyiy ullhky ihjxitilkdkpqi
9,a chosen few will become generals,tif jyupihuninkllicufjwuiqupuxtlyi


In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(dataset["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] t t h h e | | | b [PAD] [PAD] [PAD] u u n n g g [PAD] [PAD] l l l l [PAD] o o o | | w w a a s s | | [PAD] [PAD] [PAD] p l l [PAD] e s s s s [PAD] n n t t [PAD] l l l y y | | [PAD] s s s i i t t t [PAD] u u u u u [PAD] [PAD] [PAD] [PAD] a a t t [PAD] e e d d | | [PAD] n n e e a a r | | t t h e e | | s s h h [PAD] [PAD] [PAD] [PAD] o o r r r r | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'